In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time
import statsmodels.formula.api as sm

In [22]:
df = pd.read_csv('final_race_df.csv')
#df = pd.read_csv('final_bike_df.csv')

In [23]:
df = df.drop(df.columns[0], axis=1)
df.columns

Index([u'user_id', u'race_period', u'resting_heart_rate', u'hrv_rmssd', u'time_in_bed', u'latency', u'rem_sleep_duration', u'slow_wave_sleep_duration', u'light_sleep_duration', u'wake_duration', u'cycles_count', u'score', u'recovery_score', u'user_2439', u'user_2456', u'user_2458', u'user_2465', u'user_2466', u'user_2468', u'user_2469', u'user_2473', u'user_2508', u'seconds', u'FP_5K', u'Wisco_8K', u'Brown_8K', u'VCP_8K', u'FP_10K'], dtype='object')

In [45]:
df.head()

,user_id,race_period,resting_heart_rate,hrv_rmssd,time_in_bed,latency,rem_sleep_duration,slow_wave_sleep_duration,light_sleep_duration,wake_duration,cycles_count,score,recovery_score,user_2439,user_2456,user_2458,user_2465,user_2466,user_2468,user_2469,user_2473,user_2508,seconds,FP_5K,Wisco_8K,Brown_8K,VCP_8K,FP_10K
0,828,1,45.180328,0.059197,32983054.4242,1609090.75758,4505000.00000,3321818.18182,20104090.9091,5053636.36364,7.000000,80.984848,65.426230,0,0,0,0,0,0,0,0,0,1529.2,0,0,0,0,0
1,828,2,47.145833,0.055524,35072882.5625,1245231.19792,3633750.00000,4436562.50000,21996562.5000,5001874.94792,5.437500,91.906250,63.802083,0,0,0,0,0,0,0,0,0,1552.3,0,0,1,0,0
2,828,3,45.916667,0.074007,35219680.7292,1261142.54167,1525625.00000,7692187.50000,19688750.0000,6312812.50000,3.364583,87.156250,56.385417,0,0,0,0,0,0,0,0,0,1582.5,0,0,0,1,0
3,828,4,44.663366,0.078605,42391598.3267,1041284.36634,3231386.13861,8411584.15842,21082277.2277,9663564.35644,6.584158,96.257426,70.940594,0,0,0,0,0,0,0,0,0,1932.7,0,0,0,0,1
4,828,5,45.400000,0.066477,35584066.6000,1155980.93333,2920000.00000,7196000.00000,19642000.0000,5788000.00000,5.333333,92.866667,58.733333,0,0,0,0,0,0,0,0,0,1617.6,0,0,0,1,0


In [48]:
result = sm.ols(formula="seconds~ hrv_rmssd + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=df).fit()
#result = sm.ols(formula="seconds~ hrv_rmssd", data=df).fit()
#result = sm.ols(formula="seconds~ resting_heart_rate", data=df).fit()

#from pandas.stats.plm import PanelOLS
#groups = final_df.groupby(['user_id', 'race_period'])
#reg = PanelOLS(y=groups['seconds_unstd'], x=groups['hrv_rmssd'], time_effects=True)

In [49]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     118.1
Date:                Fri, 15 Jan 2016   Prob (F-statistic):           1.00e-16
Time:                        16:14:56   Log-Likelihood:                -158.95
No. Observations:                  36   AIC:                             347.9
Df Residuals:                      21   BIC:                             371.6
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1539.8726     32.677     47.124      0.000      1471.916  1607.829
hrv_rmssd   -214.5063    470.778     -0.456      0.653     -1193.543   764.530
user_2456    -41.6944     18.361     -2.271      0.034       -79.878    -3.511
user_2458     65.5947     37.300      1.759      0.093       -11.976   143.165
user_2465    -29.8553     24.207     -1.233      0.231       -80.196    20.486
user_2466     49.0151     38.865      1.261      0.221       -31.810   129.840
user_2468    -33.2016     17.406     -1.908      0.070       -69.398     2.995
user_2469     11.0848     21.701      0.511      0.615       -34.045    56.214
user_2473    -57.6303     31.641     -1.821      0.083      -123.432     8.171
user_2508     77.1230     29.479      2.616      0.016        15.817   138.429
FP_5K       -652.3537     38.876    -16.780      0.000      -733.202  -571.506
Wisco_8K      11.9941     15.879      0.755      0.458       -21.027    45.016
Brown_8K      -7.4677     22.819     -0.327      0.747       -54.922    39.987
VCP_8K        83.4421     12.236      6.819      0.000        57.996   108.888
FP_10K       427.1248     16.569     25.779      0.000       392.668   461.581
==============================================================================
Omnibus:                        1.200   Durbin-Watson:                   2.599
Prob(Omnibus):                  0.549   Jarque-Bera (JB):                0.446
Skew:                           0.217   Prob(JB):                        0.800
Kurtosis:                       3.331   Cond. No.                         127.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""